In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('points.csv')

In [3]:
df.columns = [f'p{i}' for i in range(43)]

In [4]:
y = df['p42']
X = df.drop('p42', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [6]:
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train)
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()
X_train_np.shape, y_train_np.shape

((8793, 42), (8793,))

In [7]:
model = keras.Sequential(
    [
        layers.Dense(42, activation='relu'),
        layers.Dense(21, activation='relu'),
        layers.Dense(1)
    ]
)

model.compile(
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

model.fit(X_train, y_train, batch_size = 32, epochs = 100, verbose=2)

Epoch 1/100
275/275 - 3s - loss: 13.2698 - accuracy: 0.7714 - 3s/epoch - 10ms/step
Epoch 2/100
275/275 - 1s - loss: 2.4702 - accuracy: 0.8307 - 754ms/epoch - 3ms/step
Epoch 3/100
275/275 - 1s - loss: 1.6457 - accuracy: 0.8643 - 748ms/epoch - 3ms/step
Epoch 4/100
275/275 - 1s - loss: 1.2589 - accuracy: 0.8818 - 704ms/epoch - 3ms/step
Epoch 5/100
275/275 - 1s - loss: 1.2152 - accuracy: 0.8896 - 678ms/epoch - 2ms/step
Epoch 6/100
275/275 - 1s - loss: 1.0098 - accuracy: 0.8973 - 701ms/epoch - 3ms/step
Epoch 7/100
275/275 - 1s - loss: 0.9048 - accuracy: 0.9026 - 693ms/epoch - 3ms/step
Epoch 8/100
275/275 - 1s - loss: 1.0630 - accuracy: 0.8945 - 702ms/epoch - 3ms/step
Epoch 9/100
275/275 - 1s - loss: 1.0915 - accuracy: 0.8941 - 710ms/epoch - 3ms/step
Epoch 10/100
275/275 - 1s - loss: 0.9903 - accuracy: 0.9023 - 701ms/epoch - 3ms/step
Epoch 11/100
275/275 - 1s - loss: 0.8110 - accuracy: 0.9130 - 702ms/epoch - 3ms/step
Epoch 12/100
275/275 - 1s - loss: 0.6912 - accuracy: 0.9166 - 707ms/epoch -

In [8]:
model.evaluate(X_test, y_test, batch_size = 32, verbose=2)

92/92 - 1s - loss: 0.1264 - accuracy: 0.9597 - 574ms/epoch - 6ms/step


[0.12635822594165802, 0.9597406983375549]

In [10]:
model.save('thumbs_up_model.keras')

In [2]:
model = tf.keras.models.load_model('thumbs_up_model.keras')
tf.saved_model.save(model, 'thumbs_up_saved_model')
model.summary()


INFO:tensorflow:Assets written to: thumbs_up_saved_model\assets


INFO:tensorflow:Assets written to: thumbs_up_saved_model\assets


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 42)                1806      
                                                                 
 dense_1 (Dense)             (None, 21)                903       
                                                                 
 dense_2 (Dense)             (None, 1)                 22        
                                                                 
Total params: 2731 (10.67 KB)
Trainable params: 2731 (10.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import cv2 
import numpy as np 
import mediapipe as mp
import time
import math
import pyautogui
import keyboard
from mediapipe_pose_py.im_geometry import midpoint, distance
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model('thumbs_up_model.keras')

mp_hands = mp.solutions.hands
hands_model = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

capture = cv2.VideoCapture(0)

size = pyautogui.size()

screenx = size.width
screeny = size.height

a = (int(0.01 * screenx), int(0.65 * screeny))
        
def get_hand_landmarks(hls, screenx, screeny):

    hl_list = []
    for hl in hls:
        hl_list.append({
                            'x': hl.x * screenx,
                            'y': hl.y * screeny,
                            'vis': hl.visibility
                        })
                        
    return hl_list

def predict_hand(hl_list, model):
    landmarks_array = np.array([[hl['x'], hl['y']] for hl in hl_list]).flatten()
    landmarks_array = landmarks_array.reshape(1,42)
    pred = model.predict(landmarks_array)
    if pred > 0.5:
        gesture = 'thumbs up'
    else:
        gesture = 'none'
    return gesture

while capture.isOpened():
    ret, frame = capture.read()
    if ret:
        frame = cv2.resize(frame, (screenx, screeny))

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands_model.process(image)
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                hl_list = get_hand_landmarks(hand_landmarks.landmark, screenx, screeny)
        else:
            hl_list = []

        if len(hl_list) < 21:
            gest = 'none'
        else:

            gest = predict_hand(hl_list, model)
        
        flip_image = cv2.flip(image,1)

        cv2.putText(flip_image, f"Predict: {gest}", a, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow("DrawingCV2", flip_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
    else:
        break

capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 64ms/step


C:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 [==============================] - 0s 16ms/step
